In [ ]:
import copy
import numpy as np
from scipy.stats import ks_2samp
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from Team_Win_Analysis_Runner import *
from pathlib import Path
from scipy.stats import ks_2samp
from Team_Win_Analysis_Runner import *
from pathlib import Path
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
folder=Path("Two dimensions")
Mapping=dict()
folder.mkdir(parents=True, exist_ok=True)
for (Tau,Error_Bound) in [(0.1,0.1),(0.2,0.1),(0.3,0.1),(0.4,0.1),(0.2,0.02),(0.2,0.06),(0.2,0.14)]:
    N_experts=4 #Number of experts
    N_iterations=10000 #Number of iterations
    N_players=1000
    tau=Tau
    Entry_fee=25
    Platform_fee_fraction=0.2
    correlation = 0.1
    Analytic_Error_Bound=Error_Bound
    configs = {
        "Equi-mean_Equivariance":{
            "N_experts": N_experts,
            "Means_Lognormal": [500, 500, 500, 500],
            "Std_devs_Lognormal": [60, 60, 60, 60],
            "N_players": N_players,
            "tau": tau,
            "Entry_fee": Entry_fee,
            "Platform_fee_fraction": Platform_fee_fraction,
            "correlation": correlation,
            "Analytic_Error_Bound": Analytic_Error_Bound
        },
        "Unequal-mean_Equivariance":{
            "N_experts": N_experts,
            "Means_Lognormal": [440, 473, 517, 550],
            "Std_devs_Lognormal":[60, 60, 60, 60],
            "N_players": N_players,
            "tau": tau,
            "Entry_fee": Entry_fee,
            "Platform_fee_fraction": Platform_fee_fraction,
            "correlation": correlation,
            "Analytic_Error_Bound": Analytic_Error_Bound
        },
        "Equi-mean_Unequal-std":
        {
            "N_experts": N_experts,
            "Means_Lognormal": [500, 500, 500, 500],
            "Std_devs_Lognormal":[60, 30, 60, 30],
            "N_players": N_players,
            "tau": tau,
            "Entry_fee": Entry_fee,
            "Platform_fee_fraction": Platform_fee_fraction,
            "correlation": correlation,
            "Analytic_Error_Bound": Analytic_Error_Bound
        },
        "Unequal-mean_Unequal-std":
        {
            "N_experts": N_experts,
            "Means_Lognormal": [440, 473, 517, 550],
            "Std_devs_Lognormal":[60, 30, 60, 30],
            "N_players": N_players,
            "tau": tau,
            "Entry_fee": Entry_fee,
            "Platform_fee_fraction": Platform_fee_fraction,
            "correlation": correlation,
            "Analytic_Error_Bound": Analytic_Error_Bound
        }
    }
    L={}
    for Config in configs.keys():
        N_experts = configs[Config]["N_experts"]
        Means_Lognormal = configs[Config]["Means_Lognormal"]
        Std_devs_Lognormal = configs[Config]["Std_devs_Lognormal"]
        N_players = configs[Config]["N_players"]
        tau = configs[Config]["tau"]
        Entry_fee = configs[Config]["Entry_fee"]
        Platform_fee_fraction = configs[Config]["Platform_fee_fraction"]
        correlation = configs[Config]["correlation"]
        Analytic_Error_Bound = configs[Config]["Analytic_Error_Bound"]
        L[Config]=Expert_Team_Selection(
            N_experts=N_experts,
            N_players=N_players,
            tau=tau,
            Entry_fee=Entry_fee,
            Platform_fee_fraction=Platform_fee_fraction,
            Means_Lognormal=Means_Lognormal,
            Std_devs_Lognormal=Std_devs_Lognormal,
            correlation=correlation,
            Analytic_error_bound=Analytic_Error_Bound
        )
        L[Config]={"Team_Selector":L[Config],"Highest Probability Team":int(np.argmax(np.array(L[Config].win_prob_team)))+1}       
    # Define the output directory
    Mapping[(Tau,Error_Bound)]=L    
    Winnings={}
    for Configuration in configs.keys():
        Winnings[Configuration]=L[Configuration]["Team_Selector"].run_simulation(Total_iterations=10000)
    # Save the results
    results = {
        "configs": configs,
        "Winnings": Winnings
    }
    
    # Define the output directory
    output_dir = Path(f"{folder}/Simulation_Results_Tau={Tau}_Error_Bound={Error_Bound}")
    output_dir.mkdir(parents=True, exist_ok=True)
    # Define the output file path
    with pd.ExcelWriter(f"{output_dir}/Simulation_Output.xlsx", engine="openpyxl") as writer:
        for sheet_name, df in Winnings.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    input_dir = output_dir
    data=pd.read_excel(f"{input_dir}/Simulation_Output.xlsx", sheet_name=None)
    Export_summary = {}
    Log_odds = {}
    for Config in data.keys():
        K = data[Config]
        Stat = pd.DataFrame()
        log = pd.DataFrame()
        
        # Winnings stats
        for i in range(1, N_experts + 1):
            l = K[f"Winnings_{i}"][K[f"Winnings_{i}"] != 0].describe().round(3)
            l.loc["n_zeros"] = N_iterations - l["count"]
            l.loc["Mean (including zeros)"] = K[f"Winnings_{i}"].mean()
            l.loc["Selection Ratio (Weighted) (Analytical/Random)"]="NA"
            Stat[f"Expert {i} Winnings (Non-zero)"] = l

        # Total selections
        

        # Analytical
        for i in range(1, N_experts + 1):
            l = K[f"Analytical_{i}"].describe().round(3)
            l.loc["n_zeros"] = "NA"
            l.loc["Mean (including zeros)"] = "NA"
            l.loc["Selection Ratio (Weighted) (Analytical/Random)"]="NA"
            Stat[f"Analytical_{i}"] = l

        # Random
        for i in range(1, N_experts + 1):
            l = K[f"Random_{i}"].describe().round(3)
            l.loc["n_zeros"] = "NA"
            l.loc["Mean (including zeros)"] = "NA"
            l.loc["Selection Ratio (Weighted) (Analytical/Random)"]="NA"
            Stat[f"Random_{i}"] = l

        # Log-odds (really, log of ratio between means)
        for i in range(1, N_experts + 1):
            ratio = (Stat.loc["mean", f"Analytical_{i}"] / Stat.loc["mean", f"Random_{i}"])*(1-Tau)/Tau
            #log_ratio = np.log(ratio)
            log[f"Expert {i}"] = pd.Series(ratio)
        for i in range(1, N_experts + 1):
            l = K[f"Total_{i}"].describe().round(3)
            l.loc["n_zeros"] = "NA"
            l.loc["Mean (including zeros)"] = "NA"
            l.loc["Selection Ratio (Weighted) (Analytical/Random)"] = float(log[f"Expert {i}"].iloc[0])
            Stat[f"Total_Sel_{i}"] = l

        ordered_cols = []

    # Group all winnings together
        for i in range(1, N_experts + 1):
            ordered_cols.append(f"Expert {i} Winnings (Non-zero)")

        # Then all totals
        for i in range(1, N_experts + 1):
            ordered_cols.append(f"Total_Sel_{i}")

        # Then all analytical
        for i in range(1, N_experts + 1):
            ordered_cols.append(f"Analytical_{i}")

        # Then all random
        for i in range(1, N_experts + 1):
            ordered_cols.append(f"Random_{i}")

        # Apply the reordering
        Stat = Stat[ordered_cols]
        Export_summary[Config] = Stat
    with pd.ExcelWriter(f"{input_dir}/Summary.xlsx", engine="openpyxl") as writer:
        for sheet_name, df in Export_summary.items():
            df.to_excel(writer, sheet_name=sheet_name, index=True)
    for Team_set_type in data.keys():
        directory = Path(f"{input_dir}/Plots")
        directory.mkdir(parents=True, exist_ok=True)
        
        fig, axs = plt.subplots(3, 2, figsize=(16, 18))
        fig.suptitle(f'Expert Team Distributions for {Team_set_type} (Error Bound: {Error_Bound}, Tau: {Tau})', fontsize=16)

        # 1. Winnings
        ax = axs[0, 0]
        ax.set_title('Winnings (Zeros not included)')
        for i in range(1, N_experts + 1):
            series = data[Team_set_type][f"Winnings_{i}"]
            series = series[series > 0]
            series.plot(kind='kde', label=f'Expert {i}', alpha=0.8, ax=ax)
        ax.set_xlabel("Winnings")
        ax.grid(True)
        ax.legend()
        ax = axs[0, 1]
        ax.set_title('Winnings (Zeros included)')
        for i in range(1, N_experts + 1):
            series = data[Team_set_type][f"Winnings_{i}"]
            #series = series[series > 0]
            series.plot(kind='kde', label=f'Expert {i}', alpha=0.8, ax=ax)
        ax.set_xlabel("Winnings")
        ax.grid(True)
        ax.legend()
        # 2. Total Selections
        ax = axs[1,0]
        ax.set_title('Total Selections')
        for i in range(1, N_experts + 1):
            series = data[Team_set_type][f"Total_{i}"]
            #series = series[series > 0]
            series.plot(kind='kde', label=f'Expert {i}', alpha=0.8, ax=ax)
        ax.set_xlabel("Selections")
        ax.grid(True)
        ax.legend()

        # 3. Analytical Selections
        ax = axs[1, 1]
        ax.set_title('Analytical Selections')
        for i in range(1, N_experts + 1):
            series = data[Team_set_type][f"Analytical_{i}"]
            #series = series[series > 0]
            series.plot(kind='kde', label=f'Expert {i}', alpha=0.8, ax=ax)
        ax.set_xlabel("Selections")
        ax.grid(True)
        ax.legend()

        # 4. Random Selections
        ax = axs[2, 0]
        ax.set_title('Random Selections')
        for i in range(1, N_experts + 1):
            series = data[Team_set_type][f"Random_{i}"]
            #series = series[series > 0]
            series.plot(kind='kde', label=f'Expert {i}', alpha=0.8, ax=ax)
        ax.set_xlabel("Selections")
        ax.grid(True)
        ax.legend()
        fig.delaxes(axs[2,1])
        # Final layout and save
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.savefig(f"{directory}/Summary_{Team_set_type}.png", bbox_inches="tight")
        plt.close()
    P_value={}
    for Team_set_type in data.keys():
        Stat={}
        directory = input_dir
        for i in range(1, N_experts + 1):
            # Extract the winnings data for the two groups
            Stat[f"Expert_{i}"] = {}
            for j in ["Total", "Analytical", "Random"]:
                data1 = data[Team_set_type][f"{j}_{i}"][data[Team_set_type][f"Winnings_{i}"] > 0]
                data2 = data[Team_set_type][f"{j}_{i}"][data[Team_set_type][f"Winnings_{i}"] == 0]
                
                # Perform the two-sample KS test
                statistic, p_value = ks_2samp(data1, data2)
                Stat[f"Expert_{i}"][j] = p_value
        P_value[Team_set_type] = pd.DataFrame(Stat)
        with pd.ExcelWriter(f"{directory}/P_value.xlsx", engine="openpyxl") as writer:
            for sheet_name, df in P_value.items():
                df.to_excel(writer, sheet_name=sheet_name, index=True)
    

100%|██████████| 1000/1000 [00:01<00:00, 586.33it/s]


In [ ]:
# Step 1: Load Data
Error_Bounds = [0.02, 0.06, 0.1, 0.14]
Tau = 0.2
Data = {}

for Error in Error_Bounds:
    input_dir = Path(f"Two Dimensions/Simulation_Results_Tau={Tau}_Error_Bound={Error}")
    Data[Error] = pd.read_excel(f"{input_dir}/Summary.xlsx", sheet_name=None)
    for sheet in Data[Error].keys():
        Data[Error][sheet] = Data[Error][sheet].set_index("Unnamed: 0")

# Step 2: Setup
config_names = list(configs.keys())

# Nice color palette
nice_colors = [
    "#636EFA", "#EF553B", "#00CC96", "#AB63FA",
    "#FFA15A", "#19D3F3", "#FF6692", "#B6E880"
]
color_map = {cfg: nice_colors[i % len(nice_colors)] for i, cfg in enumerate(config_names)}

# Step 3: Create Subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=[
        "Mean Winnings vs Error Bound",
        "Selection Ratio vs Error Bound"
    ],
    horizontal_spacing=0.1
)

# Step 4: Add Traces
for config in config_names:
    Best = Mapping[(Tau,Error_Bounds[0])][config]["Highest Probability Team"]
    label = f"{config} (Best: Team {Best})"

    winnings = [
        Data[err][config][f"Expert {Best} Winnings (Non-zero)"].loc['Mean (including zeros)']
        for err in Error_Bounds
    ]
    ratios = [
        Data[err][config][f"Total_Sel_{Best}"].loc['Selection Ratio (Weighted) (Analytical/Random)']
        for err in Error_Bounds
    ]

    # Plot: Mean Winnings
    fig.add_trace(
        go.Scatter(
            x=Error_Bounds,
            y=winnings,
            mode='lines+markers',
            name=label,
            line=dict(color=color_map[config], width=3),
            marker=dict(size=8),
            legendgroup=config,
            hovertemplate=f"{label}<br>Error Bound=%{{x}}<br>Winnings=%{{y:.2f}}<extra></extra>"
        ),
        row=1, col=1
    )

    # Plot: Selection Ratio
    fig.add_trace(
        go.Scatter(
            x=Error_Bounds,
            y=ratios,
            mode='lines+markers',
            name=label,
            line=dict(color=color_map[config], width=3, dash='dot'),
            marker=dict(size=8),
            legendgroup=config,
            showlegend=False,  # only show once
            hovertemplate=f"{label}<br>Error Bound=%{{x}}<br>Sel. Ratio=%{{y:.2f}}<extra></extra>"
        ),
        row=1, col=2
    )

# Step 5: Layout
fig.update_layout(
    title=f"<b>Error Bound Sensitivity Analysis (Tau = {Tau})</b>",
    title_x=0.5,
    template="plotly_white",
    width=1150,
    height=520,
    #hovermode="x unified",
    font=dict(size=14, family="Arial"),
    margin=dict(l=60, r=20, t=70, b=50),
    legend=dict(
        orientation="v",
        x=1.02,
        y=1,
        bgcolor='rgba(0,0,0,0)',
        bordercolor='rgba(0,0,0,0)'
    )
)

# Step 6: Axes
fig.update_xaxes(
    title_text="Error Bound",
    tickmode="array",
    tickvals=Error_Bounds,
    ticktext=[str(e) for e in Error_Bounds],
    row=1, col=1
)
fig.update_yaxes(title_text="Mean Winnings", row=1, col=1)

fig.update_xaxes(
    title_text="Error Bound",
    tickmode="array",
    tickvals=Error_Bounds,
    ticktext=[str(e) for e in Error_Bounds],
    row=1, col=2
)
fig.update_yaxes(title_text="Selection Ratio", row=1, col=2)

# Step 7: Save + Show
output_dir = Path("Two Dimensions")
output_dir.mkdir(parents=True, exist_ok=True)
fig.write_html(str(output_dir / "Error_Bound_Comparison_Tau=0.2.html"))
fig.show()

In [ ]:
# Step 1: Load Data
Taus = [0.1, 0.2, 0.3, 0.4]
Error = 0.1
Data = {}

for Tau in Taus:
    input_dir = Path(f"Two Dimensions/Simulation_Results_Tau={Tau}_Error_Bound={Error}")
    Data[Tau] = pd.read_excel(f"{input_dir}/Summary.xlsx", sheet_name=None)
    for sheet in Data[Tau].keys():
        Data[Tau][sheet] = Data[Tau][sheet].set_index("Unnamed: 0")

# Step 2: Setup
config_names = list(configs.keys())
nice_colors = [
    "#636EFA", "#EF553B", "#00CC96", "#AB63FA",
    "#FFA15A", "#19D3F3", "#FF6692", "#B6E880"
]
color_map = {cfg: nice_colors[i % len(nice_colors)] for i, cfg in enumerate(config_names)}

# Step 3: Create Subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=["Mean Winnings vs Tau", "Selection Ratio vs Tau"],
    horizontal_spacing=0.1
)

# Step 4: Add Traces
for config in config_names:
    best_teams = []
    winnings = []
    ratios = []

    for tau in Taus:
        best = Mapping[(tau,Error)][config]["Highest Probability Team"]
        best_teams.append(str(best))
        winnings.append(Data[tau][config][f"Expert {best} Winnings (Non-zero)"].loc['Mean (including zeros)'])
        ratios.append(Data[tau][config][f"Total_Sel_{best}"].loc['Selection Ratio (Weighted) (Analytical/Random)'])

    best_team_str = " → ".join(best_teams)  # Unicode arrow
    label = f"{config} (Best Teams: {best_team_str})"

    # Plot: Mean Winnings
    fig.add_trace(
        go.Scatter(
            x=Taus,
            y=winnings,
            mode='lines+markers',
            name=label,
            line=dict(color=color_map[config], width=3),
            marker=dict(size=8),
            legendgroup=config,
            hovertemplate=f"{label}<br>Tau=%{{x}}<br>Winnings=%{{y:.2f}}<extra></extra>"
        ),
        row=1, col=1
    )

    # Plot: Selection Ratio
    fig.add_trace(
        go.Scatter(
            x=Taus,
            y=ratios,
            mode='lines+markers',
            name=label,
            line=dict(color=color_map[config], width=3, dash='dot'),
            marker=dict(size=8),
            legendgroup=config,
            showlegend=False,
            hovertemplate=f"{label}<br>Tau=%{{x}}<br>Sel. Ratio=%{{y:.2f}}<extra></extra>"
        ),
        row=1, col=2
    )

# Step 5: Layout
fig.update_layout(
    title=f"<b>Tau Sensitivity Analysis (Error Bound = {Error})</b>",
    title_x=0.5,
    template="plotly_white",
    width=1150,
    height=520,
    hovermode="closest",
    font=dict(size=14, family="Arial"),
    margin=dict(l=60, r=20, t=70, b=50),
    legend=dict(
        orientation="v",
        x=1.02,
        y=1,
        bgcolor='rgba(0,0,0,0)',
        bordercolor='rgba(0,0,0,0)'
    )
)

# Step 6: Axes
fig.update_xaxes(
    title_text="Tau",
    tickmode="array",
    tickvals=Taus,
    ticktext=[str(t) for t in Taus],
    row=1, col=1
)
fig.update_yaxes(title_text="Mean Winnings", row=1, col=1)

fig.update_xaxes(
    title_text="Tau",
    tickmode="array",
    tickvals=Taus,
    ticktext=[str(t) for t in Taus],
    row=1, col=2
)
fig.update_yaxes(title_text="Selection Ratio", row=1, col=2)

# Step 7: Save + Show
output_dir = Path("Two Dimensions")
output_dir.mkdir(parents=True, exist_ok=True)
fig.write_html(str(output_dir / f"Tau_Comparison_ErrorBound={Error}.html"))
fig.show()

In [12]:
Mapping

{(0.1,
  0.1): {'Equi-mean_Equivariance': {'Team_Selector': <Team_Win_Analysis_Runner.Expert_Team_Selection at 0x131361010>,
   'Highest Probability Team': 1}, 'Unequal-mean_Equivariance': {'Team_Selector': <Team_Win_Analysis_Runner.Expert_Team_Selection at 0x135e549f0>,
   'Highest Probability Team': 4}, 'Equi-mean_Unequal-std': {'Team_Selector': <Team_Win_Analysis_Runner.Expert_Team_Selection at 0x135e54910>,
   'Highest Probability Team': 1}, 'Unequal-mean_Unequal-std': {'Team_Selector': <Team_Win_Analysis_Runner.Expert_Team_Selection at 0x135e54ad0>,
   'Highest Probability Team': 4}},
 (0.2,
  0.1): {'Equi-mean_Equivariance': {'Team_Selector': <Team_Win_Analysis_Runner.Expert_Team_Selection at 0x136f59710>,
   'Highest Probability Team': 3}, 'Unequal-mean_Equivariance': {'Team_Selector': <Team_Win_Analysis_Runner.Expert_Team_Selection at 0x136f5a430>,
   'Highest Probability Team': 4}, 'Equi-mean_Unequal-std': {'Team_Selector': <Team_Win_Analysis_Runner.Expert_Team_Selection at 0x